In [ ]:
import ipywidgets
import batoid
import numpy as np
import ipyvolume as ipv
from functools import lru_cache
import galsim
import os

In [ ]:
@lru_cache
def get_constellations_xyz():
    from astropy.io import fits
    try:
        xyz = fits.getdata("constellations.fits")
    except:    
        from astroquery.simbad import Simbad

        def ten(s):
            ss = s.split()
            h = float(ss[0])
            sign = +1
            if h < 0:
                sign = -1
                h *= -1
            m = float(ss[1])
            s = float(ss[2])
            return sign * (h + m/60 + s/3600)        
        
        Simbad.add_votable_fields('typed_id')
        HIPset = set()
        with open("constellationship.fab") as f:
            lines = f.readlines()
        for line in lines:
            HIPset.update([int(s) for s in line.split()[2:]])
        HIPlist = list(HIPset)
        table = Simbad.query_objects(
            [f"HIP {s}" for s in HIPlist]
        )
        table['HIPID'] = HIPlist

        xs = []
        ys = []
        zs = []
        for line in lines:
            xs.append(np.nan)
            ys.append(np.nan)
            zs.append(np.nan)
            prev_second = -1
            endpoints = iter(line.split()[2:])
            for first in endpoints:
                second = next(endpoints)
                first = int(first)
                second = int(second)

                secondrow = table[np.nonzero(table['HIPID'] == int(second))]
                ra1 = np.deg2rad(15*ten(secondrow['RA'][0]))
                dec1 = np.deg2rad(ten(secondrow['DEC'][0]))
                x1 = np.cos(ra1)*np.cos(dec1)
                y1 = np.sin(ra1)*np.cos(dec1)
                z1 = np.sin(dec1)

                if first == prev_second:
                    # just append new second
                    xs.append(x1)
                    ys.append(y1)
                    zs.append(z1)
                else:                    
                    firstrow = table[np.nonzero(table['HIPID'] == int(first))]
                    ra0 = np.deg2rad(15*ten(firstrow['RA'][0]))
                    dec0 = np.deg2rad(ten(firstrow['DEC'][0]))
                    x0 = np.cos(ra0)*np.cos(dec0)
                    y0 = np.sin(ra0)*np.cos(dec0)
                    z0 = np.sin(dec0)
                    xs.extend([np.nan, x0, x1])
                    ys.extend([np.nan, y0, y1])
                    zs.extend([np.nan, z0, z1])
                prev_second = second
        xyz = np.array([xs, ys, zs])
        fits.writeto("constellations.fits", xyz)
    return xyz

In [ ]:
class RubinCSApp:
    def __init__(self, sky_dist=15000):
        self.sky_dist = sky_dist
        self.lat = -30.2446
        self.fiducial_telescope = batoid.Optic.fromYaml("LSST_r.yaml")
        
        # widget variables
        self.clip_horizon = False
        self.lst = 0
        self.rtp = 0
        self.alt = 45
        self.az = 45
        self.thx = 0
        self.thy = 0

        # Scatters
        self.constellations = self._constellations_widget()
        self.telescope = self._telescope_widget()
        self.azimuth_ring = self._azimuth_ring_widget()
        # self.elevation_bearings = self._elevation_bearings_widget()
        
        # Controls
        self.alt_widget = ipywidgets.FloatText(value=45.0, step=1.0, description='alt (deg)')
        self.az_widget = ipywidgets.FloatText(value=45.0, step=1.0, description='az (deg)')
        self.rtp_widget = ipywidgets.FloatText(value=0.0, step=1.0, description='RTP (deg)')
        self.thx_widget = ipywidgets.FloatText(value=0.0, step=0.1, description='Field x (deg)')
        self.thy_widget = ipywidgets.FloatText(value=0.0, step=0.1, description='Field y (deg)')
        self.lst_widget = ipywidgets.FloatText(value=0.0, step=0.1, description='LST (hr)')
        self.horizon_widget = ipywidgets.Checkbox(value=self.clip_horizon, description='horizon')
        
        # observe
        self.alt_widget.observe(self.update_alt, 'value')
        self.az_widget.observe(self.update_az, 'value')
        self.rtp_widget.observe(self.update_rtp, 'value')
        self.thx_widget.observe(self.update_thx, 'value')
        self.thy_widget.observe(self.update_thy, 'value')
        self.lst_widget.observe(self.update_lst, 'value')
        self.horizon_widget.observe(self.update_horizon, 'value')

        self.update_constellations()
        self.update_telescope()
        
        self.scatters = [
            self.telescope,
            self.constellations, 
            self.azimuth_ring,
            # self.elevation_bearings
        ]
        
        self.controls = [
            self.alt_widget,
            self.az_widget,
            self.rtp_widget,
            self.thx_widget, 
            self.thy_widget, 
            self.lst_widget, 
            self.horizon_widget
        ]
    
    def _constellations_xyz(self, lst):
        ctf = batoid.CoordTransform(
            batoid.globalCoordSys,
            batoid.CoordSys(
                (0, 0, 0),
                batoid.RotZ(lst) @ batoid.RotY(-np.deg2rad(90-self.lat))
            )
        )
        return ctf.applyForwardArray(*get_constellations_xyz())        
    
    def _constellations_widget(self):
        x, y, z = self._constellations_xyz(0.0)
        return ipv.Scatter(
            x=x, y=y, z=z,
            color="blue",
            visible_lines=True, 
            color_selected=None, 
            size_selected=1, 
            size=0, 
            connected=True, 
            visible_markers=False,
            cast_shadow=True,
            receive_shadow=True
        )
    
    def _telescope_xyz(self, alt, az, rtp):
        telescope = self.fiducial_telescope
        telescope = telescope.withGlobalShift([0, 0, 3.53])  # Height of M1 vertex above azimuth ring
        telescope = telescope.withLocallyRotatedOptic("LSSTCamera", batoid.RotZ(np.deg2rad(rtp)))
        telescope = telescope.withLocalRotation(batoid.RotZ(np.deg2rad(90-az)))
        telescope = telescope.withLocalRotation(batoid.RotX(np.deg2rad(90-alt)), rotOrigin=[0, 0, 5.425], coordSys=batoid.globalCoordSys)
        return telescope.get3dmesh()
    
    def _telescope_widget(self):
        x, y, z = self._telescope_xyz(self.alt, self.az, self.rtp)
        return ipv.Scatter(
            x=x, y=y, z=z,
            color="white",
            visible_lines=True, 
            color_selected=None, 
            size_selected=1, 
            size=0, 
            connected=True, 
            visible_markers=False,
            cast_shadow=True,
            receive_shadow=True
        )
    
    def _azimuth_ring_widget(self):        
        th = np.linspace(0, 2*np.pi, 100)
        x_, y_ = np.cos(th), np.sin(th)
        xs = []
        ys = []
        zs = []
        for d in [-0.1, 0.1]:
            for r in [4.5, 5.0]:
                x = x_*r
                y = y_*r
                z = np.full_like(x, d)
                xs.append(x)
                ys.append(y)
                zs.append(z)
                xs.append([np.nan])
                ys.append([np.nan])
                zs.append([np.nan])
        xs = np.hstack(xs)
        ys = np.hstack(ys)
        zs = np.hstack(zs)
        return ipv.Scatter(
            x=xs, y=ys, z=zs,
            color="white",
            visible_lines=True, 
            color_selected=None, 
            size_selected=0,
            size=0, 
            connected=True, 
            visible_markers=False,
            cast_shadow=True,
            receive_shadow=True
        )            
            
#     def _elevation_bearings_widget(self):
#         th = np.linspace(0, 2*np.pi, 100)
#         x_, y_ = np.cos(th), np.sin(th)
#         xs = []
#         ys = []
#         zs = []    
#         for d in [4.4, 4.5]:
#             for r in [1.5, 1.75]:
#                 x = -d
#                 y = x_*r
#                 z = y_*r            
#                 z += 5.425  # height of elevation axis above azimuth ring
#                 x, y = np.cos(np.pi/2-az)*x-np.sin(np.pi/2-az)*y, np.sin(np.pi/2-az)*x+np.cos(np.pi/2-az)*y

#                 # ipv.plot(x, y, z, color='white')
#                 # ipv.plot(-x, -y, z, color='white')
            
    def update_alt(self, change):
        self.alt = change['new']
        self.update_telescope()        
    
    def update_az(self, change):
        self.az = change['new']
        self.update_telescope()        

    def update_rtp(self, change):
        self.rtp = change['new']
        self.update_telescope()        
    
    def update_lst(self, change):
        self.lst = change['new']
        self.update_constellations()        

    def update_thx(self, change):
        self.thx = change['new']

    def update_thy(self, change):
        self.thy = change['new']

    def update_horizon(self, change):
        self.clip_horizon = not self.clip_horizon
        self.update_constellations()

    def update_constellations(self):
        x, y, z = self._constellations_xyz(self.lst)*self.sky_dist
        if self.clip_horizon:
            w = z<0
            x[w] = np.nan
            y[w] = np.nan
            z[w] = np.nan
        self.constellations.x = x
        self.constellations.y = y
        self.constellations.z = z
        
    def update_telescope(self):
        x, y, z = self._telescope_xyz(self.alt, self.az, self.rtp)
        self.telescope.x = x
        self.telescope.y = y
        self.telescope.z = z        

In [ ]:
app = RubinCSApp(sky_dist=20)

fig = ipv.Figure(width=800, height=500)
fig.camera.far = 100000
fig.style = ipv.styles.dark
fig.style['box'] = dict(visible=False)
fig.style['axes']['visible'] = False
fig.xlim = (-10, 10)
fig.ylim = (-10, 10)
fig.zlim = (-10, 10)

fig.scatters = app.scatters

ipywidgets.HBox([
    fig, ipywidgets.VBox(app.controls)
])